In [1]:
spatialSample="spider" #p16 or spider. Key.

In [2]:
import scanpy as sc
import numpy as np
import pandas as pd
import tangram as tg

#This takes a few minutes.

In [3]:
adata_st=sc.read_h5ad(f"/home/hezhou/_Projects/25.02.27_Jielin_Senescence/25.04.07_prepareData/_Data/dataSpatial_{spatialSample}.h5ad")
adata_sc=sc.read_h5ad("/home/hezhou/_Projects/25.02.27_Jielin_Senescence/25.04.07_prepareData/_Data/dataSnRNASeq.h5ad")

#This takes a moment.

In [4]:
adata_st.X.shape #Cell by gene.

(1420, 19465)

In [5]:
adata_sc.X.shape #Cell by gene.

(44573, 32285)

In [6]:
# sc.tl.rank_genes_groups(adata_sc,groupby="celltype",use_raw=False)

#WARNING: It seems you use rank_genes_groups on the raw count data. Please logarithmize your data before calling rank_genes_groups.

In [7]:
adata_sc2=adata_sc.copy() #Create unlinked copy.

In [8]:
sc.pp.log1p(adata_sc2)

In [9]:
# print(adata_sc2.X)

In [10]:
sc.tl.rank_genes_groups(adata_sc2,groupby="celltype",use_raw=False)

#This takes a minute or so.

In [11]:
markers_df=pd.DataFrame(adata_sc2.uns["rank_genes_groups"]["names"]).iloc[0:100,:] #Take top 100 marker genes per cell type.

In [12]:
markers_df

,Cardiomyocytes,Dentritic cells,Endothelial cells,Epicardial cells,Fibroblasts,Immune-B cells,Immune-T cells,Macrophages,Pericytes,Smooth muscle cells
0,Ttn,Dock10,Ptprb,Slit3,Col8a1,Bank1,Skap1,Dock2,Dlc1,Kcnab1
1,Ryr2,Tbc1d8,Cdh13,Efna5,Gpc6,Arhgap15,Dock2,Fyb,Prkg1,Prkg1
2,Sorbs1,Dock2,Flt1,Ncam1,Rbms3,Ripor2,Ptprc,Zeb2,Notch3,Ctnna3
3,Tnnt2,Wdfy4,Pecam1,Zfpm2,Tenm3,Aff3,Gm2682,Ptprc,Pdgfrb,Myh11
4,Rbm20,Flt3,Ptprm,Wt1,Col5a2,Bach2,Arhgap15,Myo1f,Gpc6,Dmd
...,...,...,...,...,...,...,...,...,...,...
95,Myl2,Ap1s3,Nova2,Magi2,Adamts6,B3gnt5,Gm26740,St18,Meg3,Tbx2
96,Cdh13,Gm15987,Dgkh,mt-Co2,Tmem45a,Stap1,Apbb1ip,Cyth4,Epas1,Cyp7b1
97,Grm1,Prex1,Dock6,Gm12840,Ext1,Ly86,Pdcd4,Gng2,Adap2,Gucy1b1
98,Gm35330,Birc3,Nos3,B830012L14Rik,Svep1,Sema4d,Cd96,Tnfrsf11a,Serpine2,Rgs6


In [13]:
markers=list(np.unique(markers_df.melt().value.values))

In [14]:
len(markers) #791 marker genes.

791

In [15]:
tg.pp_adatas(adata_sc,adata_st,genes=markers) #741 training genes.

INFO:root:739 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:14817 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.


In [16]:
ad_map=tg.map_cells_to_space(adata_sc,adata_st,mode="cells") #https://github.com/broadinstitute/Tangram/blob/master/tangram/mapping_utils.py

#This takes about an hour.

INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 739 genes and rna_count_based density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.455, KL reg: 0.087
Score: 0.730, KL reg: 0.001
Score: 0.743, KL reg: 0.000
Score: 0.745, KL reg: 0.000
Score: 0.746, KL reg: 0.000
Score: 0.747, KL reg: 0.000
Score: 0.747, KL reg: 0.000
Score: 0.747, KL reg: 0.000
Score: 0.747, KL reg: 0.000
Score: 0.747, KL reg: 0.000


INFO:root:Saving results..


In [17]:
ad_map

AnnData object with n_obs × n_vars = 44573 × 1420
    obs: 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'sample', 'celltype'
    var: 'nCount_Spatial', 'nFeature_Spatial', 'uniform_density', 'rna_count_based_density'
    uns: 'train_genes_df', 'training_history'

In [18]:
ad_map.X

array([[5.8516601e-08, 1.5426836e-07, 1.2818792e-08, ..., 4.6364869e-07,
        3.7822037e-08, 1.3830071e-07],
       [7.6312013e-08, 7.7902564e-07, 8.3796841e-08, ..., 7.7367524e-08,
        7.1963899e-07, 4.3224259e-07],
       [2.8321119e-07, 5.1866698e-07, 7.0473277e-07, ..., 2.6279326e-07,
        5.3189478e-07, 5.7051035e-05],
       ...,
       [1.8275840e-07, 1.7329128e-07, 1.4010314e-07, ..., 7.7689279e-08,
        9.9298653e-08, 3.6828943e-08],
       [4.3316194e-07, 3.3896142e-06, 3.0901015e-06, ..., 1.6859352e-06,
        1.4324729e-06, 1.7233929e-06],
       [1.2208190e-07, 5.4340512e-07, 1.5945912e-07, ..., 1.0043542e-07,
        1.4200047e-07, 1.1216030e-06]], dtype=float32)

In [19]:
tg.project_cell_annotations(ad_map,adata_st,annotation="celltype")

INFO:root:spatial prediction dataframe is saved in `obsm` `tangram_ct_pred` of the spatial AnnData.


In [20]:
adata_st.obsm

AxisArrays with keys: tangram_ct_pred

In [21]:
adata_st.obsm["tangram_ct_pred"]

,Fibroblasts,Endothelial cells,Immune-T cells,Macrophages,Cardiomyocytes,Smooth muscle cells,Immune-B cells,Pericytes,Dentritic cells,Epicardial cells
AACACGTTGATACCGC-1,21.575557,11.362850,0.300845,13.691135,3.622011,0.857189,0.000938,0.431049,0.000196,0.000424
AACAGGCCATTGTCAC-1,21.724646,10.175591,0.582837,8.643933,2.236152,0.251013,0.029757,3.464691,0.119388,1.934554
AACATACTAGCCGAAG-1,7.819905,6.446111,0.206656,7.797962,1.741139,0.280650,0.233445,1.558793,0.045873,0.000380
AACATACTCCACAGTT-1,17.184962,8.163787,0.143732,1.723932,2.376367,0.000296,0.239935,4.901139,0.149791,0.000401
AACATATGCACTTCTA-1,3.525843,0.243817,0.126217,2.299476,0.000306,0.056645,0.000670,0.213007,0.232431,2.657453
...,...,...,...,...,...,...,...,...,...,...
TGTTCGTGGCGTCGTG-1,10.684135,5.501500,0.340479,6.118828,1.156107,0.135641,0.226921,0.006188,0.000345,2.014797
TGTTGCCAGTCGCCTG-1,11.547256,3.733791,0.299955,2.681159,1.652480,0.000189,0.181930,3.913997,0.185239,1.648569
TGTTGGCCTGTAGCGG-1,11.456638,3.194937,0.000749,2.350287,0.000280,0.000192,0.000536,0.298550,0.000318,1.624168
TGTTGGTGCGGAATCA-1,16.751082,11.639593,0.181805,12.398883,3.065862,0.170423,0.243697,11.750787,0.446098,0.427050


In [22]:
np.sum(adata_st.obsm["tangram_ct_pred"],axis=1) #Row sums.

AACACGTTGATACCGC-1    51.842193
AACAGGCCATTGTCAC-1    49.162562
AACATACTAGCCGAAG-1    26.130913
AACATACTCCACAGTT-1    34.884341
AACATATGCACTTCTA-1     9.355864
                        ...    
TGTTCGTGGCGTCGTG-1    26.184941
TGTTGCCAGTCGCCTG-1    25.844566
TGTTGGCCTGTAGCGG-1    18.926656
TGTTGGTGCGGAATCA-1    57.075282
TGTTGGTGGACTCAGG-1    20.461198
Length: 1420, dtype: float64

In [23]:
np.sum(adata_st.obsm["tangram_ct_pred"]) #Column sums.

Fibroblasts            18689.001289
Endothelial cells       8759.000366
Immune-T cells           430.000011
Macrophages            10181.000528
Cardiomyocytes          1931.000007
Smooth muscle cells      355.000005
Immune-B cells           282.000007
Pericytes               2703.000052
Dentritic cells          263.000003
Epicardial cells         980.000015
dtype: float64

In [24]:
np.sum(np.sum(adata_st.obsm["tangram_ct_pred"])) #Total sum. 44573.

np.float64(44573.00228174399)

Tangram plotting function doesn't work.

In [25]:
adata_st.obsm["tangram_ct_pred"].to_csv(f"/home/hezhou/_Projects/25.02.27_Jielin_Senescence/25.04.07_spatialCellTypes/1.4_resultTangram/{spatialSample}.csv")